In [158]:
from network_model import *
from pymarkowitz import *
import sys

In [159]:
GAMBLES = generate_gambles(20, left=1.3, right=1.5)
GAMBLES.append({"outcomes":[1.1, 0.0], "probs":[1.0, 0.0]})

In [160]:
gamble_averages  = np.zeros((len(GAMBLES)))
gamble_variances = np.zeros((len(GAMBLES)))

# compute expected value and variance of each gamble
for i,g in enumerate(GAMBLES):
    gamble_averages[i]  = np.average(g["outcomes"], weights=g["probs"])
    gamble_variances[i] = np.average((g["outcomes"]-gamble_averages[i])**2, weights=g["probs"])

In [161]:
GAMBLE_PRIOR_SAMPLES = np.zeros((100, 21))
for i,g in enumerate(GAMBLES):
    GAMBLE_PRIOR_SAMPLES[:,i] = np.random.choice(g["outcomes"], 100, p=g["probs"]) - 1

In [166]:
GAMBLES_MU  = np.mean(GAMBLE_PRIOR_SAMPLES-1, axis=0)
GAMBLES_COV = np.cov(GAMBLE_PRIOR_SAMPLES-1, rowvar=False)

In [167]:
GAMBLES_MU.shape

(21,)

In [168]:
PortOpt = Optimizer(GAMBLES_MU, GAMBLES_COV)
PortOpt.objective_options()

{'efficient_frontier': <Signature (w, aversion)>,
 'equal_risk_parity': <Signature (w)>,
 'min_correlation': <Signature (w)>,
 'min_volatility': <Signature (w)>,
 'min_variance': <Signature (w)>,
 'min_skew': <Signature (w)>,
 'min_kurt': <Signature (w)>,
 'min_moment': <Signature (w)>,
 'max_return': <Signature (w)>,
 'max_diversification': <Signature (w)>,
 'max_sharpe': <Signature (w, risk_free)>,
 'min_beta': <Signature (w)>,
 'max_treynor': <Signature (w, risk_free)>,
 'max_jenson_alpha': <Signature (w, risk_free, market_return)>,
 'inverse_volatility': <Signature (leverage)>,
 'inverse_variance': <Signature (leverage)>,
 'equal_weight': <Signature (leverage)>,
 'market_cap_weight': <Signature (leverage)>}

In [171]:
PortOpt.add_objective("efficient_frontier", aversion=100)
PortOpt.add_constraint("weight", weight_bound=(0,1), leverage=1)
PortOpt.solve()

PortOpt.weight_sols

array([2.70e-03, 0.00e+00, 6.00e-04, 1.34e-02, 7.50e-03, 3.91e-02,
       1.04e-02, 1.24e-02, 2.18e-02, 3.90e-03, 0.00e+00, 2.83e-02,
       0.00e+00, 5.20e-03, 2.30e-03, 8.10e-03, 5.00e-04, 3.10e-03,
       3.45e-02, 1.23e-02, 7.94e-01])

In [65]:
%%timeit

mu      = gamble_averages[:-1]
cov     = np.diag(gamble_variances[:-1])
cov_inv = np.linalg.inv(cov)
ones    = np.ones((len(mu),))
w       = cov_inv @ ((mu - 1.1) * ones)
w       = w / (ones @ w)

41.1 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [66]:
mu      = gamble_averages[:-1]
cov     = np.diag(gamble_variances[:-1])
cov_inv = np.linalg.inv(cov)
ones    = np.ones((len(mu),))
w       = cov_inv @ ((mu - 1.1) * ones)
w       = w / (ones @ w)

In [67]:
w

array([0.03924483, 0.09689386, 0.01586957, 0.08614252, 0.03282974,
       0.10430803, 0.02148364, 0.00408726, 0.04128261, 0.13067661,
       0.03779979, 0.05377645, 0.02563166, 0.09449049, 0.02139221,
       0.02108917, 0.08317395, 0.05070515, 0.0354152 , 0.00370724])

In [68]:
gamble_averages

array([1.17963861, 1.32670732, 1.1333368 , 1.28500321, 1.17963134,
       1.29986717, 1.13835076, 1.10781717, 1.18207648, 1.36334994,
       1.18990083, 1.24451449, 1.15720731, 1.31497708, 1.1447725 ,
       1.14209657, 1.28101464, 1.20431859, 1.19706401, 1.10706462,
       1.1       ])

In [69]:
x = np.concatenate([w, np.array([0])])
x

array([0.03924483, 0.09689386, 0.01586957, 0.08614252, 0.03282974,
       0.10430803, 0.02148364, 0.00408726, 0.04128261, 0.13067661,
       0.03779979, 0.05377645, 0.02563166, 0.09449049, 0.02139221,
       0.02108917, 0.08317395, 0.05070515, 0.0354152 , 0.00370724,
       0.        ])

In [73]:
%%timeit
PortOpt.add_objective("efficient_frontier", aversion=0.5)
PortOpt.add_constraint("weight", weight_bound=(0,1), leverage=1)
PortOpt.solve()

13.3 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
